<a href="https://colab.research.google.com/github/kamnsv/datawagon2/blob/main/DataWagon_2_3_WagonWizards.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Окружение

In [1]:
#@title Импорт библиотек
import os
import pyarrow.parquet as pq
import pandas as pd

# Датасет

In [15]:
#@title Общий датасет

path = 'train_1'
# Список файлов Parquet
file_paths = [x for x in os.listdir(path) if x.endswith('.parquet')]

# Чтение данных из каждого файла
dfs = [pq.read_table(path + os.sep + fname).to_pandas() for fname in file_paths]

# Объединение данных
df = pd.concat(dfs, ignore_index=True)

# Вывод общего DataFrame
df

,wagnum,model,rod_id,gruz,cnsi_gruz_capacity,cnsi_volumek,tara,date_build,srok_sl,zavod_build,...,road_id_rem,st_id_rem,fr_class,skoroport,naval,nasip,naliv,openvagons,soprovod,smerz
0,26318.0,12-600-04,1.0,682,682,85.0,240,1992-12-25,2022-04-27,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28344.0,12-132,1.0,700,700,88.0,240,2003-08-12,2024-12-24,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8099.0,11-286,0.0,670,670,138.0,270,1995-08-31,2027-10-01,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,33350.0,12-9850-02,1.0,750,750,90.0,248,2014-10-27,2047-02-05,19.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5308.0,11-276,0.0,680,680,122.0,260,1995-09-17,2027-09-28,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12769394,NaN,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaT,NaT,NaN,...,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
12769395,NaN,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaT,NaT,NaN,...,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
12769396,NaN,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaT,NaT,NaN,...,NaN,NaN,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
12769397,NaN,<NA>,NaN,<NA>,<NA>,<NA>,<NA>,NaT,NaT,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Оценка метрики

In [4]:
from train_1.metrics_f1 import calc_f1_score
path_train = r"./"
target = pd.read_csv(path_train +'/train_1/target/y_train.csv').convert_dtypes()

# подготовка презсказания. Нужно убрать, когда будут другие данные.
pred_target = target[['wagnum']]                               # номера вагонов для предсказания взяты из имеющегося файла
month_to_predict = pd.to_datetime('2022-12-01')       # дата предсказания
pred_target['current_date'] = month_to_predict                 # месяц, на который нужно предсказание
pred_target['target_month'] = 1                                # заполняю одним числом
pred_target['target_day'] = pred_target['target_month']                 # заполняю тем же числом
target_path = './train_1/prediction/target_predicton.csv'
pred_target.drop_duplicates(subset = 'wagnum').to_csv(target_path, index=False)

# таргет по прогноза выбытия вагонов в ПР на месяц и на 10 дней
true_target_path = './train_1/prediction/target_predicton_true.csv'
target.drop_duplicates(subset = 'wagnum').to_csv(true_target_path, index=False)

# оценим насколько хорошо удалось предсказать выбытие вагонов  по месяцу и по 10 дням
calc_f1_score( true_target_path, target_path,)


/tmp/ipykernel_736248/2217619062.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_target['current_date'] = month_to_predict                 # месяц, на который нужно предсказание
/tmp/ipykernel_736248/2217619062.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_target['target_month'] = 1                                # заполняю одним числом


0.06047522451698957